# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [4]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [5]:
user_recos[33][0]

RecoPath(nodes=[RecoNode(type='user', entity_id=33), RecoNode(type='product', entity_id=2346), RecoNode(type='user', entity_id=2678), RecoNode(type='product', entity_id=1762)], rels=[RecoRel(in_node=RecoNode(type='user', entity_id=33), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=1762))])

# Explanation

In [6]:
# from typing import List

# from models.reco.reco_path import RecoPath


# def generate_facts(path: RecoPath):
#     facts_txt = "% Path: \n"
#     for rel in path.rels:
#         facts_txt += rel.to_facts() + "\n"
#         user = user_registry.find_by_eid(rel.in_node.entity_id)
#         product = product_registry.find_by_eid(rel.out_node.entity_id)
#         facts_txt += rating_registry.find_user_product_rating(user.uid, product.pid).to_facts() + "\n"
#     facts_txt += "% Background Knowledge: \n"
#     for node in path.nodes:
#         if node.type == "user":
#             user = user_registry.find_by_eid(node.entity_id)
#             facts_txt += user.to_facts() + "\n"
#         elif node.type == "product":
#             product = product_registry.find_by_eid(node.entity_id)
#             facts_txt += product.to_facts() + "\n"
#     return facts_txt
            

In [7]:
# print(generate_facts(user_recos[33][0]))

In [8]:
import dotenv

dotenv.load_dotenv()

True

In [9]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [10]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

reasoning_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    # repo_id="microsoft/Phi-3-mini-4k-instruct",
    **{
        "max_new_tokens": 2048,
        "temperature": 0.1,
        "top_k": 50,
        # "top_p": 0.9,
        # "repetition_penalty": 1.1
    },
)

answering_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    # repo_id="microsoft/Phi-3-mini-4k-instruct",
    **{
        "max_new_tokens": 512,
        "temperature": 0.4,
        "top_k": 30,
        # "top_p": 0.8,
        # "repetition_penalty": 1.05
    },
)

reasoning_template = """You are an expert in graph-based recommender systems.
You try to follow the following explanation goal:
1. **Transparency:** Clearly explain how the recommendation algorithm made the decision.
2. **Scrutability:** Allow the user to provide feedback if the recommendation seems incorrect.
3. **Trust:** Build user’s confidence in the recommender system.
4. **Effectiveness:** Help user make informed decisions about the recommendation.
5. **Efficiency:** Provide a quick explanation to facilitate faster decision-making.
6. **Persuasiveness:** Convince user of the relevance of the recommendation.
7. **Satisfaction:** Enhance the ease of use and overall experience of the system for the user.
Given the background knowledge: 
{context}
Explain, step by step, why the movie {product} was recommended to the user {user}.
"""

reasoning_prompt = PromptTemplate.from_template(reasoning_template)

answer_template = """Based on the reasoning provided: {reasoning}.
You are a friendly and helpful recommender system.
Give a clear and concise explanation of why the movie {product} was recommended to the user {user}:
"""
answer_prompt = PromptTemplate.from_template(answer_template)


/home/balfroim/.cache/pypoetry/virtualenvs/trail24-UZVMavSK-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful


In [11]:
reasoning_chain = reasoning_prompt | reasoning_llm
answering_chain = answer_prompt | answering_llm

In [12]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.cot_explainer import COTExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
cot_explainer = COTExplainer(registry_handler, reasoning_chain, answering_chain)

In [13]:
path = user_recos[33][1]
# cot_explainer._prepare_input(path, ["name"])
continuation, trace = cot_explainer.explain(path)

In [14]:
print("reasoning prompt:")
print(trace.reasoning_trace.prompt)
print("reasoning completion:")
print(trace.reasoning_trace.completion)
print("answer prompt:")
print(trace.answering_trace.prompt)
print("answer completion:")
print(trace.answering_trace.completion)

reasoning prompt:
You are an expert in graph-based recommender systems.
You try to follow the following explanation goal:
1. **Transparency:** Clearly explain how the recommendation algorithm made the decision.
2. **Scrutability:** Allow the user to provide feedback if the recommendation seems incorrect.
3. **Trust:** Build user’s confidence in the recommender system.
4. **Effectiveness:** Help user make informed decisions about the recommendation.
5. **Efficiency:** Provide a quick explanation to facilitate faster decision-making.
6. **Persuasiveness:** Convince user of the relevance of the recommendation.
7. **Satisfaction:** Enhance the ease of use and overall experience of the system for the user.
Given the background knowledge: 
watched(User33, Product2254)
rated(User33, Product2254, 2)
watched(User1100, Product2254)
rated(User1100, Product2254, 4)
watched(User1100, Product1359)
rated(User1100, Product1359, 5)
gender(User33, F)
age(User33, 18-24)
name(Product2254, "Omen, The (1976

In [15]:
path = user_recos[33][0]
continuation2, _ = cot_explainer.explain(path)
continuation3, _ = cot_explainer.explain(path, ["name"])

print(continuation2)
print('----------------')
print(continuation3)


User33 was recommended the movie Product1762 because they previously enjoyed movies in the same genre, such as Product123 and Product456. Additionally, User33 has a history of rating movies with similar themes highly, such as Product789 and Product101. Product1762 falls into both of these categories, making it a strong recommendation for User33.
----------------

User33 was recommended the movie Product1762 because they previously enjoyed movies in the same genre, such as Product123 and Product456. Additionally, User33 has a history of rating movies with similar themes highly, such as Product789 and Product101. Product1762 falls into both of these categories, making it a strong recommendation for User33.
